In [1]:
import re
import emoji
import nltk
from nltk.tag import pos_tag, map_tag
import pandas as pd

ModuleNotFoundError: No module named 'emoji'

In [ ]:
df = pd.read_csv("../dataset/raw_dataset.csv")
df.head()

# simple features extraction

In [ ]:
chars, question_marks, esclamation_marks, emojis, hashtags, tags, urls, pos_count  = [], [], [], [], [], [], [], []

for index, row in df.iterrows():
    tweet = row.text
    tokens = nltk.word_tokenize(tweet)
    nr_question_marks, nr_esclamation_marks = 0, 0

    # question and esclamation marks
    for token in tokens:
        if token == '?':
            nr_question_marks += 1
        if token == '!':
            nr_esclamation_marks += 1

    # emoji
    allchars = [str for str in tweet]
    lista = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    nr_emoji = len(lista)
    for c in tweet:
        if c in lista:
            tweet = tweet.replace(c, "")

    # hashtags
    try:
        nr_hashtags = len(re.findall(r"#(\w+)", tweet))
        tweet = re.sub(r"#(\w+)", "", tweet, count=nr_hashtags)
    except Exception:
        nr_hashtags = 0


    # tags
    try:
        nr_tags = len(re.findall(r" @(\w+)", tweet))
        tweet = re.sub(r" @(\w+)", "", tweet, count=nr_tags)
    except Exception:
        nr_tags = 0

    # urls
    try:
        nr_urls = len(re.findall(r"http[s]?://([a-zA-Z0-9/.]+)", tweet))
        tweet = re.sub(r"http[s]?://([a-zA-Z0-9/.]+)", "", tweet, count=nr_urls)
    except Exception:
        nr_urls = 0
        
    # special characters
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet)
    
    #POS TAGGING
    _UNIVERSAL_TAGS = (
    "VERB",
    "NOUN",
    "ADJ",
    "ADV"
    )
    
    pos = nltk.pos_tag(tweet.split())
    simplifiedPos = [(word, map_tag('en-ptb', 'universal', tag)) for word, tag in pos]
    
    universal_tags_list = list(_UNIVERSAL_TAGS)
    count = []
    for i in range(0, len(universal_tags_list)):
        count.append(0)
    
    for element in simplifiedPos:
        for index, target in enumerate(universal_tags_list):
            if target==element[1]:
                count[index] += 1
    pos_count.append(count)
    
    # no spaces
    try:
        tweet = re.sub(" ", "", tweet, len(re.findall(" ", tweet)))
    except Exception:
        "Error: spaces have not be deleted from the tweet."
    
    chars.append(len(tweet))
    question_marks.append(nr_question_marks)
    esclamation_marks.append(nr_esclamation_marks)
    emojis.append(nr_emoji)
    hashtags.append(nr_hashtags)
    tags.append(nr_tags)
    urls.append(nr_urls)

df['plain_text_len'] = chars
df['question_marks'] = question_marks
df['esclamation_marks'] = esclamation_marks
df['emojis'] = emojis
df['hashtags'] = hashtags
df['tags'] = tags
df['urls'] = urls

## Features refactoring

In [ ]:
df1 = pd.DataFrame(pos_count, columns=universal_tags_list)
df.drop(columns=["ratio", "date", "text"], inplace=True)

result = pd.concat([df, df1], axis=1, sort=False)

#rename columns
result.rename(columns = {'VERB': 'verbs', 'NOUN': 'nouns', 'ADJ' : 'adjs', 'ADV' : 'advs'}, inplace = True)

result.to_csv("../dataset/dataset.csv", index=False)

In [1]:
import pandas as pd
df = pd.read_csv("../dataset/raw_dataset.csv")

In [5]:
df["screen_name"].value_counts()

Living_Lou         220
CookGlobal         158
KitchenSanc2ary    148
Allrecipes         115
deliciousmag       109
RecipesIdeas        61
seriouseats         58
misspickledplum     26
CookingChannel      20
indianclaypot        7
FoodizShare          6
Name: screen_name, dtype: int64

In [6]:
df["screen_name"].value_counts().sum()

928

In [6]:
df[df["retweet_count"]>20]

,screen_name,date,text,retweet_count,ratio
8,CookGlobal,2020-01-23 08:02:02,This amazingly flavorful Korean Steak Sandwich...,33,0.006047
25,CookGlobal,2020-01-21 15:15:02,"If it’s cold outside, this One-Pan English Bre...",24,0.004398
27,CookGlobal,2020-01-21 13:47:21,Lemon Rice Pilaf (Nimbu Bhath) is one of my al...,25,0.004215
37,CookGlobal,2020-01-20 19:15:02,"Crêpes are so easy to make from scratch, &amp;...",21,0.003848
43,CookGlobal,2020-01-20 10:01:01,Chicken Noodle Soup is always cozy and comfort...,21,0.003848
...,...,...,...,...,...
863,Living_Lou,2020-08-19 14:07:36,Shiitake mushrooms + garlic are in this TASTY ...,26,0.001200
868,Living_Lou,2020-08-19 04:35:02,My favourite scallop recipe!\n\nIt uses scallo...,32,0.001477
871,Living_Lou,2020-08-18 21:48:05,Cauliflower - a wonderful vegetable side dish....,22,0.001016
881,Living_Lou,2020-08-18 10:19:02,Marinate fresh seasonal tomatoes in pesto to m...,22,0.001016


In [5]:
df[["retweet_count"]].mean()

retweet_count    10.294181
dtype: float64